In [1]:
#def my_likelihood():
#    return

In [2]:
from scipy.stats import uniform
##Priors Info

# k1 = uniform(1.7*10**-9,1.06*10**-9)##...?
# GA1 = uniform(1.4,3.6)
# GA2 = uniform(0,8)
# GA3 = uniform(0.5,7.5)
# d1 = uniform(229.44,954.72)
#d2 = uniform(d1 + 487.610 , 1652.93)

In [12]:
#Conversion for scaling

MeV = 1 
GeV = 1000*MeV
fm  = 1
c   = 1
metres = (10**15)*fm
km = 1000*metres
second = 2.99*(10**8)*metres
kg = (second**2)/((1.6*(10**(-13)))*((metres)**2))
G = 6.67*10**(-11)*((metres)**3)/(kg*(second)**2)
mbar = 1.66*(10**-27)*kg

import math
dp = 10**-12
mbar = 1.66*(10**-27)*kg
pi = math.pi
Rs = ((2*G*(1.98847*(10**30)*kg))/((c**2)))
K1 =  ((4*pi*(Rs**3))/(1.98847*((10**30)*kg)*(c**2)))*(GeV/(fm)**3)

#importing libraries
import numpy as np
import scipy
import matplotlib.pyplot as plot
from scipy.integrate import odeint
from scipy.stats import uniform

#SFHx
class TOV_Solver:
    def __init__(self,Pc,p1,p2,k1,k2,k3,GA1,GA2,GA3,GA1_R,GA2_R,
                 GA3_R,GA4_R,k1_R,k2_R,k3_R,k4_R,P1_Read,P2_Read,P3_Read):
        self.Pc = Pc
        self.p1 = p1
        self.p2 = p2
        self.k1 = k1
        self.k2 = k2
        self.k3 = k3 
        self.GA1 = GA1
        self.GA2 = GA2
        self.GA3 = GA3
        self.GA1_R = GA1_R
        self.GA2_R = GA2_R
        self.GA3_R = GA3_R
        self.GA4_R = GA4_R
        self.k1_R = k1_R
        self.k2_R = k2_R
        self.k3_R = k3_R
        self.k4_R = k4_R
        self.P1_Read = P1_Read
        self.P2_Read = P2_Read
        self.P3_Read = P3_Read
    
    def EoS_Core(self,p):
    
    
        condition1 = (p < self.p1)
        condition2 = (p >= self.p1) & (p < self.p2)
        condition3 = (p >= self.p2) 
    
        condition = [condition1, condition2, condition3]
        Polytrope1 = lambda p: ((p/(self.k1))**(1/self.GA1) + (p/(self.GA1-1)))
        Polytrope2 = lambda p: ((p/(self.k2))**(1/self.GA2) + (p/(self.GA2-1)))
        Polytrope3 = lambda p: ((p/(self.k3))**(1/self.GA3) + (p/(self.GA3-1)))
        
        
        function_list = [Polytrope1,Polytrope2,Polytrope3]
        
        d = (np.piecewise(p, condition, function_list))
        return (d)
    
    
    def EoS_Crust(self,p):
        
        condition1_Read = p < self.P1_Read
        condition2_Read = (p >= self.P1_Read) & (p < self.P2_Read)
        condition3_Read = (p >= self.P2_Read) & (p < self.P3_Read)
        condition4_Read = (p >= self.P3_Read) 
    
        condition_Read = [condition1_Read,condition2_Read,condition3_Read,condition4_Read]
        
        Polytrope1_Read = lambda p: ((p/self.k1_R)**(1/self.GA1_R) + (p/(self.GA1_R-1)))
        Polytrope2_Read = lambda p: ((p/self.k2_R)**(1/self.GA2_R) + (p/(self.GA2_R-1)))
        Polytrope3_Read = lambda p: ((p/self.k3_R)**(1/self.GA3_R) + (p/(self.GA3_R-1)))
        Polytrope4_Read = lambda p: ((p/self.k4_R)**(1/self.GA4_R) + (p/(self.GA4_R-1)))
    
        function_list2 = [Polytrope1_Read,Polytrope2_Read,Polytrope3_Read,Polytrope4_Read]
        
        d = (np.piecewise(p, condition_Read, function_list2))
        return (d)

    
    def P_root(self,p):
        def F(p):
            A = self.EoS_Core(p)
            B = self.EoS_Crust(p)
            return np.abs(A-B)
        Pressure_root = scipy.optimize.fsolve(F , 10**-32)
        return Pressure_root[0]     

    def EoS_Total(self,p):
        
        condition1 = p < 3.4477591364045406e-35
        condition2 = p >= 3.4477591364045406e-35
        function1 = self.EoS_Crust(p)
        function2 = self.EoS_Core(p)
        condition_list = [condition1,condition2]
        function_list  = [function1, function2]
        d = (np.piecewise(p, condition_list, function_list))
        return d
            
    def plotting(self,p):
         #p = np.linspace(10**-30,1,1000)
         plot.plot(p,self.EoS_Total(p))
                     
                                
    def R0(self,k,GA):
            dc = ((self.Pc/k))**(1/GA) + (self.Pc/(GA - 1))
            P = self.Pc - dp 
            d = ((P/k)**(1/GA) +(P/(GA - 1)))
            nc = ((self.Pc/k)**(1/GA))
            n = (((P)/k)**(1/GA))
            return  (((3/(2*pi*(3*self.Pc + dc)))*((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))))**(0.5)

    def M0(self,k,GA):
            dc = ((self.Pc/k)**(1/GA) + (self.Pc/(GA - 1)))
            P = self.Pc - dp
            d = ((P/k)**(1/GA) +(P/(GA - 1)))
            nc = (((self.Pc)/k)**(1/GA))
            n = (((P)/k)**(1/GA))
            return ((((((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))*(2*dc/(3*self.Pc + dc))))))*self.R0(k,GA)
    
        
    def f(self,x,P):
        m,r = x
        d = self.EoS_Total(P)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp]  
            
            
    def compute_MR(self):
        
       
        Ro =  self.R0(k = self.k3, GA = self.GA3)
        Mo =  self.M0(k = self.k3, GA = self.GA3)

        R = []
        M = []
        x_i = [Mo,Ro]
        
        P0 = self.Pc - dp
            
        P1 = np.linspace(P0, 1e-40, 10000000)
        solve = odeint(self.f,x_i, P1)
                          
        m = solve[:,0]
        r = solve[:,1]

        M = (m[-2])
        R = (r[-2])*(Rs/km)
        return "M:", M ,"R:", R      

In [14]:
A = TOV_Solver(Pc = 1000,
               p1 = 5.49490219e+01/GeV, 
               p2 = 8.13601224e+02/GeV, 
               k1 = 1.8250338944950222, 
               k2 = 0.6557614364343343,
               k3 = 84.32007742276306,
               GA1 = 3.85992626, 
               GA2 = 2.62551522,
               GA3 = 1.52897461,
               GA1_R = 1.58425, 
               GA2_R = 1.28733, 
               GA3_R = 0.62223,
               GA4_R = 1.35692, 
               k1_R = 6.157198765836523e-21,
               k2_R =  1.0266961939924408e-25,
               k3_R = 1.0266961939924408e-25,
               k4_R = 1.2350927097424639e-23, 
               P1_Read= 2.1445291264915162e-33,
               P2_Read= 5.332626487963225e-28,
               P3_Read = 1.7762371605771977e-27)

A.compute_MR()

C:\Users\Abhimkul\AppData\Local\Temp/ipykernel_12640/3695785514.py:81: RuntimeWarning: invalid value encountered in power
  Polytrope1_Read = lambda p: ((p/self.k1_R)**(1/self.GA1_R) + (p/(self.GA1_R-1)))
C:\Users\Abhimkul\AppData\Local\Temp/ipykernel_12640/3695785514.py:61: RuntimeWarning: invalid value encountered in power
  Polytrope1 = lambda p: ((p/(self.k1))**(1/self.GA1) + (p/(self.GA1-1)))


('M:', 1.651599208248377, 'R:', 8.672835664857466)